<a href="https://colab.research.google.com/github/sriharikrishna/siamcse21/blob/main/rosenbrock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rosenbrock function
Also known as Rosenbrock's valley or Rosenbrock's banana function, the Rosenbrock function is a performace test for optimization. The narrow long valley makes it difficult to find the minimum. 

\begin{equation}
F(x) = \sum_{i=0}^{N-1}100(x_{i+1} - x_i^2)^2 + (1 - x_i)^2.
\end{equation}

<center><img src="https://upload.wikimedia.org/wikipedia/commons/3/32/Rosenbrock_function.svg" width="40%" /></center>


### 1. Primal Function
1. Evaluates the Rosenbrock function for an arbitary input vector.

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

def rosenbrock(x):
    """
    Input: x vector of values
    Output: Result of Rosenbrock's banana function
    """
    result = jnp.sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0,axis=0)
    return result

def plot_vals(vals, grad=None):
    plt.plot(vals,label='primal input')
    if grad != None:
      plt.plot(grad,label='derivatives')
    plt.xlabel('array index')
    plt.legend()
    plt.title('')
    plt.show()

### 2. Primal Driver
1. Calls the Rosenbrock function. 
2. We have arbitarily created an array `[0.5, 0.5..., 0.5]` as its input.

In [ ]:
def fun_driver(n):
    """
    Input: n array length
    Output: Result of Rosenbrock's banana function
    """
    val = jnp.full(n, 0.5) 

    #compute the result
    result = rosenbrock(val)
    
    print("The input is", val)
    print("The result of Rosenbrock's is ", result)

In [ ]:
fun_driver(10)

### 3. First look at derivatives: `jax.grad()`
1. https://jax.readthedocs.io/en/latest/jax.html#jax.grad
2. Computes $J\cdot v$ for a function that computes a scalar value ($R^n \rightarrow R$).
3. The seed is internally set to `1.0`
   (the shape of the seed must match the primal output).   

In [ ]:
#Create a function that computes the derivatives.
#This needs to happen only once.
grad_rosenbrock = jax.grad(rosenbrock)

def grad_driver(n):
    """
    Input: n array length
    Output: Result of Rosenbrock's banana function
    """
    #create the input array
    val = jnp.full(n, 0.5) 
    
    #compute the derivatives
    result = grad_rosenbrock(val)

    plot_vals(val,grad=result)
    print("The input is", val)
    print("The grad result is ", result)

In [ ]:
grad_driver(10)

### 4. Forward mode using: `jax.jacjvp()`
1. https://jax.readthedocs.io/en/latest/jax.html#jax.jvp
2. Computes $J\cdot v$ for a function $R^n \rightarrow R^m$. 
3. `jax.jacjvp()` requires the input value for the primal function, which is the point where the derivatives are computed.
4. `jax.jacjvp()` requires a seed $v$. 
   (For forward mode the shape of the seed must match the primal input.) 
5. The code below obtains the entire Jacobian by calling `jax.jacjvp()` multiple times. It sets the input value to an arbitary value and sets the seed to the Cartesian basis vectors.

In [ ]:
def jvp_driver(n):
    """
    Input: n array length
    Output: Result of Rosenbrock's banana function
    """
    #create the input array
    val = jnp.full(n, 0.5) 

    tangents = jnp.empty([0])

    #compute the derivatives. It takes n calls!
    iden_seed = jnp.eye(n)   
    for i in range(n):
        # jax.jvp must be called once for each input and seed value
        primal_output, res = jax.jvp(rosenbrock, (val,), (iden_seed[i],))
        tangents = jnp.append(tangents, res)
    
    plot_vals(val,grad=tangents)
    print("The input is", val)
    print("The jax.jvp result is ", tangents)

In [ ]:
jvp_driver(10)

### 5. Reverse mode using: `jax.jacvjp()`
1. https://jax.readthedocs.io/en/latest/jax.html#jax.vjp
2. Uses reverse mode, returns a function (`fun_vjp` below) that computes $v \cdot J$ for a function $R^n \rightarrow R^m$.
3. `jax.jacvjp()` requires the input value for the primal function, which is the point where the derivatives are computed.
4. `fun_vjp` requires a seed $v$. For reverse mode the shape of the seed must match the primal output. We have chosen `1.0`. 
5. Exercise: See how values change as the seed changes.

In [ ]:
def vjp_driver(n):    
    """
    Input: n array length
    Output: Result of Rosenbrock's banana function
    """
    #create the input array
    val = jnp.full(n, 0.5) 

    #jax.vjp must be called once for each input value
    primals, fun_vjp = jax.vjp(rosenbrock, val)
    
    #compute the derivatives by calling fun_vjp() for the seed value
    seed = 1.0
    adj_val = fun_vjp(seed)

    plot_vals(val,grad=jnp.stack(adj_val[0]))
    print("The input is", val)
    print("The jax.vjp result is ", adj_val[0])

In [ ]:
vjp_driver(10)

### 6. Multiple Input Arguments
1. `jax.grad()` `jax.jvp()` `jax.vjp()` all support multiple arguments as *input*.


In [ ]:
def rosenbrock3(x0,x1,x2):
    """
    Computes the Rosenbrock function
    x1,x2,x3 : scalar input values
    """
    result = (100*(x1-x0**2)**2 + (1-x0)**2) + (100*(x2-x1**2)**2 + (1-x1)**2)
    return result

#Call the original Rosebrock
fun_driver(3)

result = rosenbrock3(0.5,0.5,0.5)
print("The result of rosenbrock3 is", result)


In [ ]:
def grad_driver3(x0,x1,x2):
    """
    n must be a positive integer
    """
    #Create a function that computes the derivatives
    grad_rosenbrock3 = jax.grad(rosenbrock3, argnums=(0,1,2))

    #compute the result
    result1, result2, result3 = grad_rosenbrock3(x0,x1,x2)
    print("The input is", x0,x1,x2)
    print("The grad result is ", result1, result2, result3)

#Call the original Rosebrock grad_driver
grad_driver(3)
grad_driver3(0.5,0.5,0.5)

We will have to call the `grad` function using different seed values:

$\bullet$ Initializing $\dot{x_1} = 1$ and $\dot{x_2} = 0$ $\dot{x_3} = 0$ yields $\dot{f} = \frac{\partial f}{\partial x_1}$.

$\bullet$ Initializing $\dot{x_2} = 1$ and $\dot{x_1} = 0$ $\dot{x_3} = 0$ yields $\dot{f} = \frac{\partial f}{\partial x_2}$.

$\bullet$ Initializing $\dot{x_3} = 1$ and $\dot{x_1} = 0$ $\dot{x_2} = 0$ yields $\dot{f} = \frac{\partial f}{\partial x_3}$.

In [ ]:
def jvp_driver3(x0,x1,x2):
    """
    n must be a positive integer
    """
    #compute the result
    primal, result1 = jax.jvp(rosenbrock3,(x0,x1,x2), (1.0,0.0,0.0))
    primal, result2 = jax.jvp(rosenbrock3,(x0,x1,x2), (0.0,1.0,0.0))
    primal, result3 = jax.jvp(rosenbrock3,(x0,x1,x2), (0.0,0.0,1.0))
    print("The input is", x0,x1,x2)
    print("The grad result is ", (result1), (result2), (result3))

jvp_driver3(0.5,0.5,0.5)